# Using Creators abd Degraders to Generate Galaxy Samples with Errors and Biases

This notebook demonstrates how to use a RAIL Creator to create galaxy samples, and how to use Degraders to add various errors and biases to the sample.

Note that in the parlance of the Creation Module, "degradation" is any post-processing that occurs to the "true" sample generated by the Creator's Engine. This can include adding photometric errors, applying quality cuts, introducing systematic biases, etc.

In this notebook, we will first learn how to draw samples from a RAIL Creator object.
Then we will demonstrate how to use the following RAIL Degraders:
1. [**LSSTErrorModel**](#LSSTErrorModel), which adds photometric errors
2. [**BandCut**](#BandCut), which applies cuts to the specified columns of the sample
3. [**InvRedshiftIncompleteness**](#InvRedshiftIncompleteness), which introduces sample incompleteness
4. [**LineConfusion**](#LineConfusion), which introduces spectroscopic errors

Throughout the notebook, we will show how you can chain all these Degraders together to build a more complicated degrader.
Hopefully, this will allow you to see how you can build your own degrader.

*Note on generating redshift posteriors*: regardless of what Degraders you apply, when you use a Creator to estimate posteriors, the posteriors will *always* be calculated with respect to the "true" distribution. This is the whole point of the Creation Module -- you can generate degraded samples for which we still have access to the *true* posteriors. For an example of how to calculate posteriors, see `posterior-demo.ipynb`.

In [ ]:
from pzflow.examples import get_example_flow
from rail.creation import Creator, engines
from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness, LineConfusion, BandCut
import matplotlib.pyplot as plt

## "True" Creator

First, let's make a Creator that has no degradation. We can use it to generate a "true" sample, to which we can compare all the degraded samples below.

Note: When instantiating a Creator, you must supply an "engine". This can be any object with `sample` and `get_posterior` methods. In this example, we will use a normalizing flow from the [pzflow](https://github.com/jfcrenshaw/pzflow) package. However, everything in this notebook is totally agnostic to what the underlying engine is.

In [ ]:
flowEngine = engines.FlowEngine(get_example_flow())
creator_truth = Creator(flowEngine)

In [ ]:
nsamples = int(1e5)
samples_truth = creator_truth.sample(nsamples, seed=0)
samples_truth

<a id="LSSTErrorModel"></a>
## Degrader 1: LSSTErrorModel

Now, we will demonstrate the `LSSTErrorModel`, which adds photometric errors using the model from [Ivezic et al. 2019](https://arxiv.org/abs/0805.2366). 

Let's create an error model with the default settings
(with one exception: the Flow we are using inside these Creators produces photometry with the names "u", "g", "r", etc. instead of "lsst_u", "lsst_g", "lsst_r", etc. Therefore, I pass a bandNames dictionary that translates between these two naming schemes)

In [ ]:
errorModel = LSSTErrorModel(bandNames={f"lsst_{b}": b for b in "ugrizy"})

To see the details of the model, including the default settings we are using, you can just print the model:

In [ ]:
errorModel

Now let's add this error model as a degrader and draw some samples with photometric errors.

In [ ]:
creator_w_errs = Creator(flowEngine, errorModel)
samples_w_errs = creator_w_errs.sample(nsamples, seed=0)
samples_w_errs

Notice some of the magnitudes are NaN's. These are non-detections. This means those observed magnitudes were beyond the 30mag limit that is default in `LSSTErrorModel`. 
You can change this limit and the corresponding flag by setting `magLim=...` and `ndFlag=...` in the constructor for `LSSTErrorModel`. 

Let's plot the error as a function of magnitude

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "ugrizy":
    
    # pull out the magnitudes and errors
    mags = samples_w_errs[band].to_numpy()
    errs = samples_w_errs[band+"_err"].to_numpy()
    
    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]
    
    # plot errs vs mags
    ax.plot(mags, errs, label=band)  
    
ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()

You can see that the photometric error increases as magnitude gets dimmer, just like you would expect. Notice, however, that we have galaxies as dim as magnitude 30. This is because the Flow produces a sample much deeper than the LSST 5-sigma limiting magnitudes. There are no galaxies dimmer than magnitude 30 because LSSTErrorModel sets magnitudes > 30 equal to NaN (the default flag for non-detections).

<a id="BandCut"></a>
## Degrader 2: BandCut

Recall how the sample above has galaxies as dim as magnitude 30. This is well beyond the LSST 5-sigma limiting magnitudes, so it will be useful to apply cuts to the data to filter out these super-dim samples. We can apply these cuts using the `BandCut` degrader (note that despite its name, you can use `BandCut` to apply cuts to any column in the sample, including redshift).

Let's create a Degrader that first adds photometric errors, then cuts at i<25.3, which is the LSST gold sample.

In [ ]:
def goldCutWithErrs(data, seed=None):
    # apply the error model from before
    data = errorModel(data, seed)
    # now make a cut on observed i band
    data = BandCut({"i": 25.3})(data, seed)
    return data

Now we can stick this into a Creator and draw a new sample

In [ ]:
creator_gold_w_errs = Creator(flowEngine, degrader=goldCutWithErrs)
samples_gold_w_errs = creator_gold_w_errs.sample(nsamples, seed=0)
samples_gold_w_errs

If you look at the i column, you will see there are no longer any samples with i > 25.3. You can also see that despite making the cut on the i band, there are still 100000 samples as requested. This is because after making the cut, the creator will draw more samples (and re-apply the cut) iteratively until you have as many samples as originally requested. 

One more note: it is easy to use the BandCut degrader as a SNR cut on the magnitudes. The magnitude equation is $m = -2.5 \log(f)$. Taking the derivative, we have
$$
dm = \frac{2.5}{\ln(10)} \frac{df}{f} = \frac{2.5}{\ln(10)} \frac{1}{\mathrm{SNR}}.
$$
So if you want to make a cut on galaxies above a certain SNR, you can make a cut
$$
dm < \frac{2.5}{\ln(10)} \frac{1}{\mathrm{SNR}}.
$$
For example, an SNR cut on the i band would look like this: `BandCut({"i_err": 2.5/np.log(10) * 1/SNR})`.

<a id="InvRedshiftIncompleteness"></a>
## Degrader 3: InvRedshiftIncompleteness

Next, we will demonstrate the `InvRedshiftIncompleteness` degrader. It applies a selection function, which keeps galaxies with probability $p_{\text{keep}}(z) = \min(1, \frac{z_p}{z})$, where $z_p$ is the ''pivot'' redshift. We'll use $z_p = 0.8$.

In [ ]:
def incompleteGoldWithErrs(data, seed=None):
    # apply error model and make cut on observed i band
    data = goldCutWithErrs(data, seed)
    # introduce redshift incompleteness
    data = InvRedshiftIncompleteness(0.8)(data, seed)
    return data

In [ ]:
creator_incomplete_gold_w_errs = Creator(flowEngine, degrader=incompleteGoldWithErrs)
samples_incomplete_gold_w_errs = creator_incomplete_gold_w_errs.sample(nsamples, seed=0)

Let's plot the redshift distributions of the samples we have generated so far:

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

zmin = 0
zmax = 2.5

hist_settings = {
    "bins": 50,
    "range": (zmin, zmax),
    "density": True,
    "histtype": "step",
}

ax.hist(samples_truth["redshift"], label="Truth", **hist_settings)
ax.hist(samples_gold_w_errs["redshift"], label="Gold", **hist_settings)
ax.hist(samples_incomplete_gold_w_errs["redshift"], label="Incomplete Gold", **hist_settings)
ax.legend(title="Sample")
ax.set(xlim=(zmin, zmax), xlabel="Redshift", ylabel="Galaxy density")
plt.show()

You can see that the Gold sample has significantly fewer high-redshift galaxies than the truth. This is because many of the high-redshift galaxies have i > 25.3.

You can further see that the Incomplete Gold sample has even fewer high-redshift galaxies. This is exactly what we expected from this degrader.

<a id="LineConfusion"></a>
## Degrader 4: LineConfusion

`LineConfusion` is a degrader that simulates spectroscopic errors resulting from the confusion of different emission lines.

For this example, let's use the degrader to simulate a scenario in which which 2% of [OII] lines are mistaken as [OIII] lines, and 1% of [OIII] lines are mistaken as [OII] lines. (note I do not know how realistic this scenario is!)

In [ ]:
def confusedIncompleteGoldWithErrs(data, seed=None):
    # apply error model, make cut on i band, introduce incompleteness
    data = incompleteGoldWithErrs(data, seed)
    
    # Oxygen lines (in angstroms)
    OII = 3727
    OIII = 5007
    # 2% OII -> OIII confusion
    data = LineConfusion(true_wavelen=OII, wrong_wavelen=OIII, frac_wrong=0.02)(data, seed)
    # 1% OIII -> OII confusion
    data = LineConfusion(true_wavelen=OIII, wrong_wavelen=OII, frac_wrong=0.01)(data, seed)
    
    return data

In [ ]:
creator_conf_inc_gold_w_errs = Creator(flowEngine, degrader=confusedIncompleteGoldWithErrs)
samples_conf_inc_gold_w_errs = creator_conf_inc_gold_w_errs.sample(nsamples, seed=0)

Let's plot the redshift distributions one more time

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

zmin = 0
zmax = 2.5

hist_settings = {
    "bins": 50,
    "range": (zmin, zmax),
    "density": True,
    "histtype": "step",
}

ax.hist(samples_truth["redshift"], label="Truth", **hist_settings)
ax.hist(samples_gold_w_errs["redshift"], label="Gold", **hist_settings)
ax.hist(samples_incomplete_gold_w_errs["redshift"], label="Incomplete Gold", **hist_settings)
ax.hist(samples_conf_inc_gold_w_errs["redshift"], label="Confused Incomplete Gold", **hist_settings)
ax.legend(title="Sample")
ax.set(xlim=(zmin, zmax), xlabel="Redshift", ylabel="Galaxy density")
plt.show()

You can see that the redshift distribution of this new sample is essentially identical to the Incomplete Gold sample, with small perturbations that result from the line confusion. 

However the real impact of this degrader isn't on the redshift distribution, but rather that it introduces erroneous spec-z's into the photo-z training sets! To see the impact of this effect, let's plot the true spec-z's as present in the Incomplete Gold sample, vs the spec-z's listed in the new sample with Oxygen Line Confusion.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=85)

ax.scatter(samples_incomplete_gold_w_errs["redshift"], samples_conf_inc_gold_w_errs["redshift"], 
           marker=".", s=1)

ax.set(
    xlim=(0, 2.5), ylim=(0, 2.5),
    xlabel="True spec-z (in Incomplete Gold sample)",
    ylabel="Spec-z listed in the Confused sample",
)
plt.show()

Now we can clearly see the spec-z errors! The galaxies above the line y=x are the [OII] -> [OIII] galaxies, while the ones below are the [OIII] -> [OII] galaxies.